# Problem Description

Uh-oh - you've been cornered by one of Commander Lambdas elite guards! Fortunately, you grabbed a beam weapon from an abandoned guard post while you were running through the station, so you have a chance to fight your way out. But the beam weapon is potentially dangerous to you as well as to the elite guard: its beams reflect off walls, meaning you'll have to be very careful where you shoot to avoid bouncing a shot toward yourself!

Luckily, the beams can only travel a certain maximum distance before becoming too weak to cause damage. You also know that if a beam hits a corner, it will bounce back in exactly the same direction. And of course, if the beam hits either you or the guard, it will stop immediately (albeit painfully). 

Write a function solution(dimensions, your_position, guard_position, distance) that gives an array of 2 integers of the width and height of the room, an array of 2 integers of your x and y coordinates in the room, an array of 2 integers of the guard's x and y coordinates in the room, and returns an integer of the number of distinct directions that you can fire to hit the elite guard, given the maximum distance that the beam can travel.

The room has integer dimensions [1 < x_dim <= 1250, 1 < y_dim <= 1250]. You and the elite guard are both positioned on the integer lattice at different distinct positions (x, y) inside the room such that [0 < x < x_dim, 0 < y < y_dim]. Finally, the maximum distance that the beam can travel before becoming harmless will be given as an integer 1 < distance <= 10000.

For example, if you and the elite guard were positioned in a room with dimensions [3, 2], your_position [1, 1], guard_position [2, 1], and a maximum shot distance of 4, you could shoot in seven different directions to hit the elite guard (given as vector bearings from your location): [1, 0], [1, 2], [1, -2], [3, 2], [3, -2], [-3, 2], and [-3, -2]. As specific examples, the shot at bearing [1, 0] is the straight line horizontal shot of distance 1, the shot at bearing [-3, -2] bounces off the left wall and then the bottom wall before hitting the elite guard with a total shot distance of sqrt(13), and the shot at bearing [1, 2] bounces off just the top wall before hitting the elite guard with a total shot distance of sqrt(5).


**Test cases**

Your code should pass the following test cases.
Note that it may also be run against hidden test cases not shown here.



Input: solution.solution([3,2], [1,1], [2,1], 4)

Output:     7


Input: solution.solution([300,275], [150,150], [185,100], 500)

Output:     9

# Problem solution

The number of possible directions to shoot is infite, so is clearly not a option to test direcitons blindly.
             
To determine if a shoot in the direction $v$ will hit the target $X$, possibly reflecting in the line $l$, we can determine if it hits $X$ *or* $X'$, being $X'$ the reflexion of $X$ the line $l$, as ilustrated below:

             X'    
            /      
    _______/_______l
          /\        
       v /  \      
             X      
             
The reflexion of X by the line l is given by (\cite{1}):

$$
X'=2{\frac {X\cdot l}{l\cdot l}}l-X
$$

Our reflexion lines are the border of the map, therefore this equation can be much simplified. 
After listing the reflexions of our interest points (our position and the guard's position) that lays below the distance limit, we just need to test which of the shoots (lines connecting a guard's reflexion to our position) do not pass by one of our reflexions.

I've found two similar codes that implement this idea, that differ basicly by the way of cheking if a shoot if valid (connect a guard reflexion to our position without passing by one of our reflexions): the implementation from \cite{2} do it by brute force, while the implementation from \cite{3} do it by dynamic programming \cite{4}. 
Both aproaches were tested running 100 times the two test cases given, in a computer with Intel Core i5 7200U 2.5GHz processor and 8GB DDR4 of memory. The runtime are compared in the table bellow: 

| Algorithm  | Runtime |
| ---------- | ------- |
| Solution 1 | 25.3 ms |
| Solution 2 | 66.0 ms |

# References
[1] - Wikipedia. Reflection (mathematics). Available in: https://en.wikipedia.org/wiki/Reflection_(mathematics)#Reflection_across_a_line_in_the_plane

[2] - Junaid. Github repository. Available in: https://gist.github.com/junaid1460/ab6da07a204e7f21e622dbd91632884e

[3] - Gauvain Tarbouriech. Website. Available in: https://govanify.com/post/foobar/

[4] - Thomas H. Cormen, Charles E. Leiserson, Ronald L. Rivest, and Clifford Stein. 2009. Introduction to Algorithms, Third Edition (3rd ed.). The MIT Press.

In [285]:
# Implemented by \cite{2}
import math
def generate_dist_vector(size, start, tot_length, length):
    tmp = [start]
    count = 0
    l, r = -length, tot_length - length
    for i in range(size):
        left = tmp[0]
        right = tmp[count]
        left += (l*2)
        right += (r*2)
        l, r = -r,-l
        tmp = [left] + tmp + [right]
        count += 2
    return tmp

def make_mat(vec1, vec2):
    mat = []
    count = 0
    for i in vec2:
        mat.append([])
        for j in vec1:
            mat[count].append((j,i))
        count += 1
    return mat

def translate(x, y, vec):
    mat = []
    count = 0
    for i in vec:
        mat.append([])
        for j in i:
            mat[count].append((j[0] + x, j[1] + y))
        # print mat[count]
        count += 1

    return mat

def serialize(vec):
    start = int(len(vec)/ 2) 
    elms = [vec[start][start]]
    count = 3
    start -= 1
    while( start >= 0):
        x = start
        y = start
        for j in range(1, count):
            x+= 1
            elms+= [vec[y][x]]
        for j in range(1, count):
            y+= 1
            elms+= [vec[y][x]]
        for j in range(1, count):
            x-= 1
            elms+= [vec[y][x]]
        for j in range(1, count):
            y-= 1
            elms+= [vec[y][x]]
        
        start -= 1
        count += 2
    return elms

def key(x,y):
    return format(math.atan2(x,y),'.32f')
    

def dist(x,y):
    return math.hypot(x,y)

def calc(cap,bad, distance):
    visited = {}
    l = len(cap)
    count  = 0
    for i in range(l):
        ce = cap[i]
        be = bad[i]  
        visited[key(ce[0], ce[1])] = True
        if distance - dist(be[0], be[1]) >=0 :
            k = key(be[0], be[1])
            if k not in visited:
                count += 1
                visited[k] = True
        else:
            k = key(be[0], be[1])
            visited[k] = True
    return count

def solutionBrute(dimensions, captain, badguy, distance):
    _x = 0
    _y = 1
    tx = captain[_x]
    ty = captain[_y]
    width = dimensions[0] 
    height = dimensions[1]
    mat_size = int(math.ceil(max( distance/width, distance / height))) + 1
    bad_x =  generate_dist_vector(mat_size, badguy[_x],width,badguy[_x])
    bad_y =  generate_dist_vector(mat_size, badguy[_y],height,badguy[_y])
    cap_x =  generate_dist_vector(mat_size, captain[_x],width,captain[_x])
    cap_y =  generate_dist_vector(mat_size, captain[_y],height,captain[_y])
    elms_bad =  serialize(translate(-tx, -ty, make_mat(bad_x, bad_y)))
    elms_cap =  serialize(translate(-tx, -ty, make_mat(cap_x, cap_y)))


    return calc(elms_cap, elms_bad, distance)

In [286]:
## Complete documentation in: 
## Implemented by \cite{3}

from math import atan2
from math import sqrt

def mirror_atlas(node, dimensions, distance):
    node_mirrored=[]
    for i in range(len(node)):
        points=[]
        for j in range(-(distance//dimensions[i])-1, (distance//dimensions[i]+2)):
            points.append(get_mirror(j, node[i], dimensions[i]))
        node_mirrored.append(points)
    return node_mirrored

def get_mirror(mirror, coordinates, dimensions):
    res=coordinates
    mirror_rotation=[2*coordinates, 2*(dimensions-coordinates)]
    if(mirror<0):
        for i in range(mirror, 0):
            res-=mirror_rotation[(i+1)%2]
    else:
        for i in range(mirror, 0, -1):
            res+=mirror_rotation[i%2]
    return res 

def solution(dimensions, your_position, guard_position, distance):
    mirrored = [mirror_atlas(your_position, dimensions,
        distance),mirror_atlas(guard_position, dimensions, distance)]
    res=set()
    angles_dist={}
    for i in range(0, len(mirrored)):
        for j in mirrored[i][0]:
            for k in mirrored[i][1]:
                beam=atan2((your_position[1]-k), (your_position[0]-j))
                l=sqrt((your_position[0]-j)**2 + (your_position[1]-k)**2)
                if [j,k] != your_position and distance >= l:
                    if((beam in angles_dist and angles_dist[beam] > l) or beam not in angles_dist):
                        if i == 0:
                            angles_dist[beam] = l
                        else:
                            angles_dist[beam] = l
                            res.add(beam)
    return len(res) 

In [284]:
from time import time
import gc

X = [([300,275], [150,150], [185,100], 500), ([3,2], [1,1], [2,1], 4)]*100
y = [9, 7]*100
gc.collect()

for f in [solution, solutionBrute]:
    time0 = time()
    y_hat = [f(arg[0], arg[1], arg[2], arg[3]) for arg in X]
    time1 = time()
    if y_hat != y: print ("Fail, dumb ass")
    else: print("%s ms" % ((time1 - time0)*1000))

25.355100631713867 ms
66.04504585266113 ms
